In [1]:
import os
from binance.client import Client
client = Client(api_key, api_secret)
client._get_earliest_valid_timestamp('BTCUSDT', '1m')

from datetime import datetime
import pandas as pd
import requests
from typing import *
import time

class BinanceClient:
    def __init__(self, futures=False):
        self.exchange = "BINANCE"
        self.futures = futures

        if self.futures:
            self._base_url = "https://fapi.binance.com"
        else:
            self._base_url = "https://api.binance.com"

        self.symbols = self._get_symbols()

    def _make_request(self, endpoint: str, query_parameters: Dict):
        try:
            response = requests.get(self._base_url + endpoint, params=query_parameters)
        except Exception as e:
            print("Connection error while making request to %s: %s", endpoint, e)
            return None

        if response.status_code == 200:
            return response.json()
        else:
            print("Error while making request to %s: %s (status code = %s)",
                         endpoint, response.json(), response.status_code)
            return None

    def _get_symbols(self) -> List[str]:

        params = dict()

        endpoint = "/fapi/v1/exchangeInfo" if self.futures else "/api/v3/exchangeInfo"
        data = self._make_request(endpoint, params)

        symbols = [x["symbol"] for x in data["symbols"]]

        return symbols

    def get_historical_data(self, symbol: str, interval: Optional[str] = "1m", start_time: Optional[int] = None, end_time: Optional[int] = None, limit: Optional[int] = 1500):

        params = dict()

        params["symbol"] = symbol
        params["interval"] = interval
        params["limit"] = limit

        if start_time is not None:
            params["startTime"] = start_time
        if end_time is not None:
            params["endTime"] = end_time

        endpoint = "/fapi/v1/klines" if self.futures else "/api/v3/klines"
        raw_candles = self._make_request(endpoint, params)

        candles = []

        if raw_candles is not None:
            for c in raw_candles:
                candles.append((float(c[0]), float(c[1]), float(c[2]), float(c[3]), float(c[4]), float(c[5]),))
            return candles
        else:
            return None

def ms_to_dt_utc(ms: int) -> datetime:
    return datetime.utcfromtimestamp(ms / 1000)

def ms_to_dt_local(ms: int) -> datetime:
    return datetime.fromtimestamp(ms / 1000)

def GetDataFrame(data):
    df = pd.DataFrame(data, columns=['Timestamp', "Open", "High", "Low", "Close", "Volume"])
    df["Timestamp"] = df["Timestamp"].apply(lambda x: ms_to_dt_local(x))
    df['Date'] = df["Timestamp"].dt.strftime("%d/%m/%Y")
    df['Time'] = df["Timestamp"].dt.strftime("%H:%M:%S")
    column_names = ["Date", "Time", "Open", "High", "Low", "Close", "Volume"]
    df = df.set_index('Timestamp')
    df = df.reindex(columns=column_names)

    return df

def GetHistoricalData(client, symbol, start_time, end_time, limit=1500):
    collection = []

    while start_time < end_time:
        data = client.get_historical_data(symbol, start_time=start_time, end_time=end_time, limit=limit)
        print(client.exchange + " " + symbol + " : Collected " + str(len(data)) + " initial data from "+ str(ms_to_dt_local(data[0][0])) +" to " + str(ms_to_dt_local(data[-1][0])))
        start_time = int(data[-1][0] + 1000)
        collection +=data
        time.sleep(1.1)

    return collection
    
client = BinanceClient(futures=False)
symbol = "BTCUSDT"
interval = "1h"
fromDate = int(datetime.strptime('2022-01-23', '%Y-%m-%d').timestamp() * 1000)
toDate = int(datetime.strptime('2022-01-25', '%Y-%m-%d').timestamp() * 1000)

data = GetHistoricalData(client, symbol, fromDate, toDate)
df = GetDataFrame(data)
#df.to_csv('btc_1h_data.csv')

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


BINANCE BTCUSDT : Collected 1000 initial data from 2022-01-23 00:00:00 to 2022-01-23 16:39:00
BINANCE BTCUSDT : Collected 1000 initial data from 2022-01-23 16:40:00 to 2022-01-24 09:19:00
BINANCE BTCUSDT : Collected 1000 initial data from 2022-01-24 09:20:00 to 2022-01-25 01:59:00
BINANCE BTCUSDT : Collected 1000 initial data from 2022-01-25 02:00:00 to 2022-01-25 18:39:00
BINANCE BTCUSDT : Collected 1000 initial data from 2022-01-25 18:40:00 to 2022-01-26 11:19:00
BINANCE BTCUSDT : Collected 1000 initial data from 2022-01-26 11:20:00 to 2022-01-27 03:59:00
BINANCE BTCUSDT : Collected 1000 initial data from 2022-01-27 04:00:00 to 2022-01-27 20:39:00
BINANCE BTCUSDT : Collected 1000 initial data from 2022-01-27 20:40:00 to 2022-01-28 13:19:00
BINANCE BTCUSDT : Collected 1000 initial data from 2022-01-28 13:20:00 to 2022-01-29 05:59:00
BINANCE BTCUSDT : Collected 1000 initial data from 2022-01-29 06:00:00 to 2022-01-29 22:39:00
BINANCE BTCUSDT : Collected 1000 initial data from 2022-01-2

In [16]:
def get_binance_data(api_key, pair, countdown='open', interval='1h', start='01 Jan 2022', end=None):
  client = Client(api_key=api_key)
  intervals = {
      '15m': Client.KLINE_INTERVAL_15MINUTE,
      '1h':  Client.KLINE_INTERVAL_1HOUR,      
      '4h':  Client.KLINE_INTERVAL_4HOUR,
      '1d':  Client.KLINE_INTERVAL_1DAY
  }
  interval = intervals.get(interval, '1h')
  print(f'Historical interval {interval}')
  klines = client.get_historical_klines(symbol=pair, interval=interval, start_str=start, end_str=end)
  data = pd.DataFrame(klines)
  data.columns = ['open_time','open', 'high', 'low', 'close', 'volume','close_time', 'qav','num_trades','taker_base_vol','taker_quote_vol', 'ignore']
  data.index = [pd.to_datetime(x, unit='ms').strftime('%Y-%m-%d %H:%M:%S') for x in data.open_time]
  #usecols=['open', 'high', 'low', 'close', 'volume']
  #data = data[usecols]
  data = data.astype('float')
  return data



symbol = 'BTCUSDT'
df = get_binance_data(api_key, symbol)
df.to_csv('btc_1h_data.csv')

Historical interval 1h


In [20]:
df

,open_time,open,high,low,close,volume,close_time,qav,num_trades,taker_base_vol,taker_quote_vol,ignore
2022-01-01 00:00:00,1.640995e+12,46216.93,46731.39,46208.37,46656.13,1503.33095,1.640999e+12,6.987999e+07,38608.0,806.06536,3.746216e+07,0.0
2022-01-01 01:00:00,1.640999e+12,46656.14,46949.99,46574.06,46778.14,943.81539,1.641002e+12,4.412715e+07,31872.0,491.18067,2.296327e+07,0.0
2022-01-01 02:00:00,1.641002e+12,46778.14,46928.94,46721.96,46811.77,485.16860,1.641006e+12,2.272067e+07,24364.0,206.35113,9.662607e+06,0.0
2022-01-01 03:00:00,1.641006e+12,46811.77,46916.63,46760.12,46813.20,562.88971,1.641010e+12,2.636326e+07,19882.0,248.28212,1.162938e+07,0.0
2022-01-01 04:00:00,1.641010e+12,46813.21,46887.33,46591.23,46711.05,861.88389,1.641013e+12,4.027204e+07,23357.0,397.53563,1.857540e+07,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-26 07:00:00,1.698304e+12,34615.16,34784.61,34551.50,34647.34,2098.65673,1.698307e+12,7.281145e+07,69793.0,1198.96420,4.159797e+07,0.0
2023-10-26 08:00:00,1.698307e+12,34647.35,34745.86,34499.00,34520.85,1940.94276,1.698311e+12,6.713873e+07,62795.0,986.15616,3.411179e+07,0.0
2023-10-26 09:00:00,1.698311e+12,34520.86,34585.77,34405.27,34429.23,1622.90871,1.698314e+12,5.594760e+07,53063.0,783.94296,2.702419e+07,0.0
2023-10-26 10:00:00,1.698314e+12,34429.23,34450.81,34100.64,34277.56,2720.37183,1.698318e+12,9.335226e+07,86913.0,1192.83656,4.092673e+07,0.0
